In [2]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
M = Vir.verma_module(1/2, 1/16)
d = Vir.basis()
v = M.highest_weight_vector()
m2 = (3*d[-2] - 4*d[-1]*d[-1])*v
m4 = (49*d[-2]*d[-2]  - 600*d[-2]*d[-1]*d[-1] + 144*d[-1]*d[-1]*d[-1]*d[-1]
      + 264*d[-3]*d[-1] - 36*d[-4])*v

def L_partition_vector(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return sum(row[i]*L_partition_vector(list_of_partition[i], v) for i in range(len(row)))

def my_order(partition):
    polynomial = to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial))

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def matrix_UVir_m2_m4(n):
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 4)
    matrix_m2 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition2[i], m2))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m4 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition3[i], m4))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m4)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m4(n)
    pivots = A_n.pivots()
    try:
        return to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

In [3]:
list_of_partition = [[2], [1, 1, 1, 1], [3, 1, 1, 1], [3, 3, 1], [4, 3, 1], [4, 4, 1, 1],
                     [5, 4, 1, 1, 1], [5, 5, 1, 1, 1], [6, 5, 3, 1, 1], [6, 6, 3, 1, 1],
                     [7, 6, 4, 1, 1, 1], [8, 7, 5, 3, 1, 1]]
%time mW_list_of_partition(list_of_partition[:-6])

CPU times: user 3.8 s, sys: 12.1 ms, total: 3.82 s
Wall time: 3.82 s


[d[-2]*v - 4/3*d[-1]*d[-1]*v,
 -465/256*d[-4]*v - 75/32*d[-3]*d[-1]*v + d[-1]*d[-1]*d[-1]*d[-1]*v,
 -183/512*d[-6]*v - 111/64*d[-5]*d[-1]*v - 1/8*d[-4]*d[-1]*d[-1]*v - 75/128*d[-3]*d[-3]*v + d[-3]*d[-1]*d[-1]*d[-1]*v,
 -13/20*d[-7]*v - 83/20*d[-6]*d[-1]*v - 6/5*d[-5]*d[-1]*d[-1]*v - 3/10*d[-4]*d[-3]*v + 16/15*d[-4]*d[-1]*d[-1]*d[-1]*v + d[-3]*d[-3]*d[-1]*v,
 -51/128*d[-8]*v - 37/16*d[-7]*d[-1]*v - d[-6]*d[-1]*d[-1]*v + 3/16*d[-5]*d[-3]*v + 1/8*d[-4]*d[-4]*v + d[-4]*d[-3]*d[-1]*v,
 -4203/1024*d[-10]*v - 2895/128*d[-9]*d[-1]*v - 41/3*d[-8]*d[-1]*d[-1]*v + 285/128*d[-7]*d[-3]*v - 4*d[-7]*d[-1]*d[-1]*d[-1]*v + 75/64*d[-6]*d[-4]*v + 21/8*d[-6]*d[-3]*d[-1]*v + 193/256*d[-5]*d[-5]*v + 6*d[-5]*d[-4]*d[-1]*v + 4/3*d[-5]*d[-3]*d[-1]*d[-1]*v + d[-4]*d[-4]*d[-1]*d[-1]*v]

In [4]:
%time mW_list_of_partition(list_of_partition[:-5])

CPU times: user 20 s, sys: 10.7 ms, total: 20.1 s
Wall time: 20.1 s


[d[-2]*v - 4/3*d[-1]*d[-1]*v,
 -465/256*d[-4]*v - 75/32*d[-3]*d[-1]*v + d[-1]*d[-1]*d[-1]*d[-1]*v,
 -183/512*d[-6]*v - 111/64*d[-5]*d[-1]*v - 1/8*d[-4]*d[-1]*d[-1]*v - 75/128*d[-3]*d[-3]*v + d[-3]*d[-1]*d[-1]*d[-1]*v,
 -13/20*d[-7]*v - 83/20*d[-6]*d[-1]*v - 6/5*d[-5]*d[-1]*d[-1]*v - 3/10*d[-4]*d[-3]*v + 16/15*d[-4]*d[-1]*d[-1]*d[-1]*v + d[-3]*d[-3]*d[-1]*v,
 -51/128*d[-8]*v - 37/16*d[-7]*d[-1]*v - d[-6]*d[-1]*d[-1]*v + 3/16*d[-5]*d[-3]*v + 1/8*d[-4]*d[-4]*v + d[-4]*d[-3]*d[-1]*v,
 -4203/1024*d[-10]*v - 2895/128*d[-9]*d[-1]*v - 41/3*d[-8]*d[-1]*d[-1]*v + 285/128*d[-7]*d[-3]*v - 4*d[-7]*d[-1]*d[-1]*d[-1]*v + 75/64*d[-6]*d[-4]*v + 21/8*d[-6]*d[-3]*d[-1]*v + 193/256*d[-5]*d[-5]*v + 6*d[-5]*d[-4]*d[-1]*v + 4/3*d[-5]*d[-3]*d[-1]*d[-1]*v + d[-4]*d[-4]*d[-1]*d[-1]*v,
 -291801/16384*d[-12]*v - 189243/2048*d[-11]*d[-1]*v - 16189/256*d[-10]*d[-1]*d[-1]*v + 17253/2048*d[-9]*d[-3]*v - 767/32*d[-9]*d[-1]*d[-1]*d[-1]*v - 567/1024*d[-8]*d[-4]*v - 27/8*d[-8]*d[-3]*d[-1]*v + 7437/2048*d[-7]*d[-5]*v + 35

In [5]:
%time mW_list_of_partition(list_of_partition[:-4])

CPU times: user 54.5 s, sys: 551 µs, total: 54.5 s
Wall time: 54.5 s


[d[-2]*v - 4/3*d[-1]*d[-1]*v,
 -465/256*d[-4]*v - 75/32*d[-3]*d[-1]*v + d[-1]*d[-1]*d[-1]*d[-1]*v,
 -183/512*d[-6]*v - 111/64*d[-5]*d[-1]*v - 1/8*d[-4]*d[-1]*d[-1]*v - 75/128*d[-3]*d[-3]*v + d[-3]*d[-1]*d[-1]*d[-1]*v,
 -13/20*d[-7]*v - 83/20*d[-6]*d[-1]*v - 6/5*d[-5]*d[-1]*d[-1]*v - 3/10*d[-4]*d[-3]*v + 16/15*d[-4]*d[-1]*d[-1]*d[-1]*v + d[-3]*d[-3]*d[-1]*v,
 -51/128*d[-8]*v - 37/16*d[-7]*d[-1]*v - d[-6]*d[-1]*d[-1]*v + 3/16*d[-5]*d[-3]*v + 1/8*d[-4]*d[-4]*v + d[-4]*d[-3]*d[-1]*v,
 -4203/1024*d[-10]*v - 2895/128*d[-9]*d[-1]*v - 41/3*d[-8]*d[-1]*d[-1]*v + 285/128*d[-7]*d[-3]*v - 4*d[-7]*d[-1]*d[-1]*d[-1]*v + 75/64*d[-6]*d[-4]*v + 21/8*d[-6]*d[-3]*d[-1]*v + 193/256*d[-5]*d[-5]*v + 6*d[-5]*d[-4]*d[-1]*v + 4/3*d[-5]*d[-3]*d[-1]*d[-1]*v + d[-4]*d[-4]*d[-1]*d[-1]*v,
 -291801/16384*d[-12]*v - 189243/2048*d[-11]*d[-1]*v - 16189/256*d[-10]*d[-1]*d[-1]*v + 17253/2048*d[-9]*d[-3]*v - 767/32*d[-9]*d[-1]*d[-1]*d[-1]*v - 567/1024*d[-8]*d[-4]*v - 27/8*d[-8]*d[-3]*d[-1]*v + 7437/2048*d[-7]*d[-5]*v + 35

In [6]:
%time mW_list_of_partition(list_of_partition[:-3])

CPU times: user 5min 58s, sys: 59.9 ms, total: 5min 58s
Wall time: 5min 58s


[d[-2]*v - 4/3*d[-1]*d[-1]*v,
 -465/256*d[-4]*v - 75/32*d[-3]*d[-1]*v + d[-1]*d[-1]*d[-1]*d[-1]*v,
 -183/512*d[-6]*v - 111/64*d[-5]*d[-1]*v - 1/8*d[-4]*d[-1]*d[-1]*v - 75/128*d[-3]*d[-3]*v + d[-3]*d[-1]*d[-1]*d[-1]*v,
 -13/20*d[-7]*v - 83/20*d[-6]*d[-1]*v - 6/5*d[-5]*d[-1]*d[-1]*v - 3/10*d[-4]*d[-3]*v + 16/15*d[-4]*d[-1]*d[-1]*d[-1]*v + d[-3]*d[-3]*d[-1]*v,
 -51/128*d[-8]*v - 37/16*d[-7]*d[-1]*v - d[-6]*d[-1]*d[-1]*v + 3/16*d[-5]*d[-3]*v + 1/8*d[-4]*d[-4]*v + d[-4]*d[-3]*d[-1]*v,
 -4203/1024*d[-10]*v - 2895/128*d[-9]*d[-1]*v - 41/3*d[-8]*d[-1]*d[-1]*v + 285/128*d[-7]*d[-3]*v - 4*d[-7]*d[-1]*d[-1]*d[-1]*v + 75/64*d[-6]*d[-4]*v + 21/8*d[-6]*d[-3]*d[-1]*v + 193/256*d[-5]*d[-5]*v + 6*d[-5]*d[-4]*d[-1]*v + 4/3*d[-5]*d[-3]*d[-1]*d[-1]*v + d[-4]*d[-4]*d[-1]*d[-1]*v,
 -291801/16384*d[-12]*v - 189243/2048*d[-11]*d[-1]*v - 16189/256*d[-10]*d[-1]*d[-1]*v + 17253/2048*d[-9]*d[-3]*v - 767/32*d[-9]*d[-1]*d[-1]*d[-1]*v - 567/1024*d[-8]*d[-4]*v - 27/8*d[-8]*d[-3]*d[-1]*v + 7437/2048*d[-7]*d[-5]*v + 35